In [8]:
def location_crawler(url1,r):
    import re
    import time
    from urllib.parse import urljoin
    from urllib.parse import urlparse
    import requests
    from bs4 import BeautifulSoup
    valid_url = re.compile(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)')
    soup = BeautifulSoup(r.content,'lxml')
    goto_location = soup.find('a',text=re.compile(r'Τοποθεσία|Που είμαστε|Πως θα μας βρείτε|Θα μας βρείτε|How to find us|Χάρτης|Χάρτες|πρόσβαση|Χάρτες & πρόσβαση|Χάρτης Google|^([Εε]πικοινωνία(?!.*επενδυτές).*$|[Cc]ontact [Us]?|Στοιχεία επικοινωνίας|Επικοινωνία|CONTACTS|Επικοινωνία|Επικοινωνείστε μαζί μας|INFO|ΕΠΙΚΟΙΝΩΝΙΑ|ΕΠΙΚΟΙΝΩΝΗΣΤΕ ΜΑΖΙ ΜΑΣ|ΕΠΙΚΟΙΝΩΝΙΑ|Επικοινωνια|[Cc]ontact|Contacts|CONTACT|[εΕ]πικοινωνια|ÎÏÎ¹ÎºÎ¿Î¹Î½ÏÎ½Î¯Î±|Επικοινωνία(?!.*επενδυτές).*$|Επικοινώνησε μαζί μας|^(ÎÏÎ¹ÎºÎ¿Î¹Î½ÏÎ½Î¹Î±)|ÎµÏÎ¹ÎºÎ¿Î¹Î½ÏÎ½Î¯Î±|ÅÐÉÊÏÉÍÙÍÉÁ|Πού θα μας βρείτε|[Cc]ontact|ÎÎ ÎÎÎÎÎÎ©ÎÎÎ|ΕΠΙΚ/ΝΙΑ|Επικοινωνήστε μαζί μας|Που είμαστε|ΕΠΙΚΟΙΝΩΝΙΑ|Επικοινωνια|Στοιχεία επικοινωνίας|Φόρμα Επικοινωνίας|Επικοινωνήστε μαζί μας|Επικοινωνήστε μαζί μας)'),attrs={'href' : re.compile(r'^(?!.*(javascript|mailto|Mailto))')})
    if goto_location == None:
        goto_location = soup.find(href = re.compile(r'[cC]ontact[-us]?\b|CONTACT|Forma Epikoinwnias|communication|EPIKOINONIA|katastimata|know|contact_us|epik|ΕΠΙΚΟΙΝΩΝΙΑ|stoixeia-epikoinonias|Contacts|stores|communication|epikoinia|Contact|contact|[eE]pikoin[ow]nia\b|contact|%C5%F0%E9%EA%EF%E9%ED%F9%ED%DF%E1|contac-us|epikoinwnia|where-we-are|find-us|2cont|contact|επικοινωνία|epikinonia|επικοινωνια|%ce%b5%cf%80%ce%b9%ce%ba%ce%bf%ce%b9%ce%bd%cf%89%ce%bd%ce%af%ce%b1|EPIKINONIA|contact-us|epikoinwnia|epikoinonia|Eπικοινωνία|Επικοινωνία|καταστήματα|επικοινωνια|epsilonpiiotakappaomicroniotanuomeganuiotaalpha|GrFeed|madres-oikodomikwn-ylikwn-volou.html|epikoinoneste-mazi-mas|/1/gr/8_3/8.htm'))
    if goto_location !=None and 'mailto' not in goto_location['href'] and 'Mailto' not in goto_location['href'] and 'javascript' not in goto_location['href']:
        if valid_url.search(goto_location['href']):
            location_link = goto_location['href']
        else:
            location_link = urljoin(url1,goto_location['href'])
    else:
        location_link = url1
    res = get_coordinates(location_link)
    if res == []:
        parsed = list(urlparse(url1))[1]
        parsed = re.sub('www.','',parsed)
        parsed = re.sub('.eu','',parsed)
        parsed = re.sub('.com','',parsed)
        parsed = re.sub('.gr','',parsed)
        parsed = re.sub('.biz','',parsed)
        parsed = re.sub('.net','',parsed)
        parsed = re.sub('.me','',parsed)
        parsed = re.sub('.webnode','',parsed)
        res = find_place(parsed)
        coords = res[1]
        adrr = res[0]
        tk = res[2]
    else:
        time.sleep(1.1)
        adrr,tk = geolocate(res)
        coords =','.join(res)
    result = adrr+','+coords+','+tk
    if result ==',,':
        result = tk_search(location_link)
    return result
def get_coordinates(newlink):
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.common.exceptions import StaleElementReferenceException
    import time 
    import re
    import requests
    path="C:\\Users\\User\\Desktop\\diploma thesis Data Science\\chromedriver.exe"
    driver=webdriver.Chrome(path)
    #print(newlink)
    driver.get(newlink)
    time.sleep(5)
    try:
        cards = driver.find_elements_by_xpath("//div[@id]")
    except WebDriverException:
        cards = []
    except StaleElementReferenceException:
        cards = []
    #print(cards)
    regexp = re.compile(r'http[s]?://maps.google.com/maps?')
    hrefs = []
    if cards!=[]:
        for header in cards:
            a = header.find_elements_by_xpath(".//a[@target]")
            for cc in a:
                #print(cc.get_attribute('href'))
                if cc.get_attribute('href') ==None:
                    continue
                else:
                    #print(cc.get_attribute('href'))
                    if regexp.search(cc.get_attribute('href')):
                        hrefs.append(cc.get_attribute('href'))
    if hrefs == []:
        r6 = requests.get(newlink)
        soup6 = BeautifulSoup(r6.text,'lxml')
        iframes = soup6.find_all('iframe',src=re.compile(r'http[s]?://maps.google.com/maps?'))
        if iframes == []:
            try:
                iframes22 = soup6.find_all('iframe',src=re.compile(r'google'))
                if iframes22 == []:
                    hrefs = []
                    lonlat = []
                else:
                    for frame in iframes22:
                        tmp = re.findall("\d{1,2}\.\d+",frame['src'])
                        if len(tmp) ==2:
                            hrefs.append(tmp[0]+','+tmp[1])
                        elif tmp == []:
                            hrefs = []
                        else:
                            tmp67 = []
                            for tm in tmp:
                                if (float(tm)>=31 and float(tm)<=42):
                                    tmp1 = tm
                                if (float(tm)<=29 and float(tm)>=19.86):
                                    tmp2 = tm
                            #tmp = sorted(tmp67, key=float, reverse=True)
                            #tm = [float(x) for x in tmp]
                            #tmp1 = max(tm)
                            #tm1 = [t for t in tm if t!=tmp1]
                            #tmp2 = max(tm1)
                            hrefs.append(str(tmp1)+','+str(tmp2))
                    lonlat = hrefs
            except ValueError:
                divs = soup6.find('div',src=re.compile(r'google-maps-link'))
                if divs !=None:
                    tar = divs.find('a',href=re.compile(r'http[s]?://maps.google.com/maps?'))
                    if tar !=None:
                        hrefs = []
                        hrefs.append(tar['href'])
                        driver.quit()
                        return clean_coords(hrefs)
                    else:
                        driver.quit()
                        return []
                else:
                    driver.quit()
                    return []
        else:
            for c in iframes:
                hrefs.append(c['src'])
            lonlat = clean_coords(hrefs)
    else:
        lonlat = clean_coords(hrefs)
    driver.quit()
    #lonlat = [re.sub('%2C',',',l) for l in lonlat]
    lonlat = [','.join(re.findall("\d+\.\d+",l)) for l in lonlat]
    lonlat = [re.sub('[^\d\.,]','',l).rstrip('.').lstrip('.').rstrip(',').lstrip(',') for l in lonlat]
    if lonlat == [',,,'] or lonlat == [',']:
        lonlat = []
    return lonlat
def clean_coords(hr):
    returns = list(set(hr))
    #res = [link[link.find('=')+1:link.find('&')].split(',') for link in returns]
    res = []
    for ss in returns:
        try:
            k = ss.split('ll=')[1]
            res.append(k[:k.find('&')])
        except IndexError:
            k=ss.split('q=')[1]
            res.append(k[:k.find('&')].replace('+',','))
        except IndexError:
            continue
    return res
def geolocate(coords):
    from geopy.geocoders import Nominatim
    import re
    import time
    import requests
    apkey = 'your-key'
    #geolocator = Nominatim(timeout=10)
    add = []
    pcode = []
    for c in coords:
        if c == '':
            continue
        else:
            try:
                c = re.sub('%2C',',',c)
                k = c.split(',')
                if float(k[1]) > float(k[0]):
                    c = k[1]+ ','+ k[0]
            except ValueError:
                c = re.sub('%2C','',c)
                k = c.split(',')
                if float(k[1]) > float(k[0]):
                    c = k[1]+ ','+ k[0]
            except ValueError:
                continue
            url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng='+c+'&key='+apkey
            content = requests.get(url).json()
            add.append(content['results'][0]['formatted_address'])
            p1 = content['results'][0]['address_components']
            pcode.append(p1[len(p1)-1]['short_name'])
            #location = geolocator.reverse(c,timeout=10)
            #if 'address' in location.raw.keys():
            #    add.append(location.address)
            #    add1 = location.raw['address']
            #    if 'postcode' not in add1.keys():
            #        pcode.append('')
            #    else:
            #        pcode.append(location.raw['address']['postcode'])
            time.sleep(1.2)
    retk = ''.join(pcode)
    retad = ','.join(add)
    return retad,retk
def find_place(place_name):
    from urllib.parse   import quote
    import requests
    import time
    
    api_key = 'api-key'
    fname = place_name + ' '+ 'GREECE'
    try:
        url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query='+quote(fname)+'&key='+api_key
        content = requests.get(url).json()
    except:
        time.sleep(5)
        api2 = 'api-key'
        url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query='+quote(fname)+'&key='+api2
        content = requests.get(url).json()
    if content['results'] != []:
        address = content['results'][0]['formatted_address']
        coordinates = str(content['results'][0]['geometry']['location']['lat'])+','+str(content['results'][0]['geometry']['location']['lng'])
        tk = geocoding_forplaces(coordinates)
        return [address,coordinates,tk]
    else:
        return ['','','']
def geocoding_forplaces(coordinates):
    import requests
    geocoding_api1 = 'api-key'
    url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng='+coordinates+'&key='+geocoding_api1
    content = requests.get(url).json()
    p1 = content['results'][0]['address_components']
    return p1[len(p1)-1]['short_name']
import pandas as pd
geo_data = pd.read_csv('zipcodes.txt', header = None)
tks = list(geo_data[0])
tks = [str(t) for t in tks]
def tk_search(newlink):
    import pandas as pd
    import requests
    import re
    from bs4 import BeautifulSoup
    from nltk.tokenize import RegexpTokenizer
    from collections import Counter
    r = requests.get(newlink)
    soup = BeautifulSoup(r.content,'lxml')
    texts = soup.findAll(text=True)
    visible_texts = filter(visible, texts)
    res = list(visible_texts)
    totaltext = ' '.join(res)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens2 = tokenizer.tokenize(totaltext)
    tokens3 = []
    #print(tokens2)
    found_tk = []
    for tok in tokens2:
        if tok in tks:
            found_tk.append(tok)
    if found_tk == []:
        for first, second in zip(tokens2, tokens2[1:]):
            tmpb = first+second
            if tmpb in tks:
                found_tk.append(tmpb)
    finaltk = list(set(found_tk))
    if finaltk !=[]:
        add = []
        for t in finaltk:
            add.append(geocodetk(t))
        return add[0]
    else:
        return ',,'
def geocodetk(tks):
    import requests
    apkey = 'api-key'
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address='+tks+'GREECE&key='+apkey
    r = requests.get(url).json()
    location = r['results'][0]['geometry']['location']
    coords = str(location['lat'])+','+str(location['lng'])
    addr = r['results'][0][ 'formatted_address']
    return addr+','+coords+','+tks
def visible(element):
    import re
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element)):
        return False
    return True
def check_redirects(url):
    import requests
    r = requests.get(url)
    r1 = requests.get(r.url)
    return r.url,r1
def run_me(url):
    url1,r = check_redirects(url)
    print(location_crawler(url1,r))

In [9]:
run_me('http://couniniotis.gr/')

Omirou, Egialia 251 00, Greece,38.258797,22.062338,251 00


In [11]:
run_me('http://www.veler.gr/')

Sapfous 29-35, Acharnes 136 71, Greece,38.0629975797081,23.73970891578256,136 71


In [12]:
run_me('http://hellasfrost.gr/')

EO53, Metaxades 680 10, Greece,41.4199,26.2241,680 10


In [13]:
run_me('http://www.agrimon.gr/')

Makedonomachon 9, Serres 621 25, Greece,41.081743,23.547921,621 25


In [14]:
run_me('http://www.kallas-pap.com/')

Damianou 6, Acharnes 136 71, Greece,38.079156,23.746416,136 71


In [15]:
run_me('http://www.tsianakas.gr/')

EO Volou Larisas, Volos 385 00, Greece,39.374696,22.897904,385 00


In [18]:
run_me('http://rotosal.gr/')

EO Livadias Lamias, Livadia 321 00, Greece,38.474058770973386,22.91725906207575,321 00


In [19]:
run_me('http://www.viozokat.gr/')

3ο ΧΛΜ ΕΘΝΙΚΗΣ ΟΔΟΥ ΚΑΤΕΡΙΝΗΣ - ΛΑΡΙΣΑΣ, Katerini 601 00, Greece,40.24747499999999,22.533289,601 00


In [20]:
run_me('http://www.biofresh-sa.com/')

Leof. Andrea Siggrou 18, Athina 117 42, Greece,37.967979,23.7309127,117 42


In [21]:
run_me('http://www.domissima.gr/')

Eparch.Od. Neon Moudanion - Thessalonikis, Nea Propontida 630 80, Greece,40.387301,23.038009,630 80


In [22]:
run_me('http://www.pansil.gr/')

Iera Odos ke Naxou, Egaleo 122 42, Greece,37.98822759999999,23.6920481,122 42


In [23]:
run_me('http://www.hfo.gr/')

Alamanas 1, Marousi 151 25, Greece,38.043729,23.807108,151 25


In [24]:
run_me('http://www.tsarouchas.eu/')

EO Kavalas Xanthis, Nestos 642 00, Greece,40.97531902497841,24.59798222451531,642 00


In [26]:
run_me('http://www.andrianos.gr/gr/')

Mezonos 22, Kalamata 241 00, Greece,37.032865,22.114345,241 00


In [27]:
run_me('http://www.akrodria.gr/')

Oraiokastro 570 13, Greece,40.7137773,22.9283587,57013


In [29]:
run_me('https://glaveris.gr/')

Athanasiou Diakou, Kimina 573 00, Greece,40.6156,22.6813,573 00


In [33]:
run_me('http://www.spiroskatsaros.gr/')

Voriou Ipirou 78, Athina 104 44, Greece,37.997651,23.718259,104 44
